In [7]:
import json
import pprint as pp
import pandas as pd
import tqdm

In [8]:
with open("persons_parsed_with_region.json", 'r') as f:
    persons = json.load(f)

In [10]:
def family_comp(pid, persons):
    person = persons[pid]
    res = {
        "name": person["name"],
        "id": person["id"],
        "region": person["region"],
        "years": {}
    }
    
    for declaration in person["sections"]:
        year = declaration["main"]["year"]
        res["years"][year] = {"personal_income": 0, "family_income": 0, "childrens_income": 0,\
                                                    "personal_sq": 0, "family_sq": 0, "childrens_sq": 0,\
                                                    "personal_veh": 0, "family_veh": 0, "childrens_veh": 0,\
                                                    "ap_sq": 0, "land_sq": 0}
        for income in declaration["incomes"]:
            if income["relative"] == None:
                res["years"][year]["personal_income"] += float(income["size"]) if float(income["size"]) != None else 0
            elif income["relative"]["id"] == 2:
                res["years"][year]["family_income"] += float(income["size"]) if float(income["size"]) != None else 0
            else:
                res["years"][year]["childrens_income"] += float(income["size"]) if float(income["size"]) != None else 0
                
                
        for estate in declaration["real_estates"]:
            if estate["relative"] == None:
                res["years"][year]["personal_sq"] += estate["square"] if estate["square"] != None else 0
                if estate["type"]["id"] == 3 or estate["type"]["id"] == 4 :
                    res["years"][year]["ap_sq"] += estate["square"] if estate["square"] != None else 0
                else:
                    res["years"][year]["land_sq"] += estate["square"] if estate["square"] != None else 0
            elif estate["relative"]["id"] == 2:
                res["years"][year]["family_sq"] += estate["square"] if estate["square"] != None else 0
                if estate["type"]["id"] == 3 or estate["type"]["id"] == 4 :
                    res["years"][year]["ap_sq"] += estate["square"] if estate["square"] != None else 0
                else:
                    res["years"][year]["land_sq"] += estate["square"] if estate["square"] != None else 0
            else:
                res["years"][year]["childrens_sq"] += estate["square"] if estate["square"] != None else 0
                if estate["type"]["id"] == 3 or estate["type"]["id"] == 4 :
                    res["years"][year]["ap_sq"] += estate["square"] if estate["square"] != None else 0
                else:
                    res["years"][year]["land_sq"] += estate["square"] if estate["square"] != None else 0
                
        for veh in declaration["vehicles"]:
            if veh["relative"] == None:
                res["years"][year]["personal_veh"] += 1
            elif veh["relative"]["id"] == 2:
                res["years"][year]["family_veh"] += 1
            else:
                res["years"][year]["childrens_veh"] += 1
                
        res["years"][year]["total_income"] = res["years"][year]["personal_income"] \
                                            + res["years"][year]["childrens_income"] \
                                            + res["years"][year]["family_income"]
        
        res["years"][year]["total_sq"] = res["years"][year]["personal_sq"] \
                                            + res["years"][year]["childrens_sq"] \
                                            + res["years"][year]["family_sq"]
        
        res["years"][year]["total_veh"] = res["years"][year]["personal_veh"] \
                                            + res["years"][year]["childrens_veh"] \
                                            + res["years"][year]["family_veh"]
    return res

In [11]:
family_results = {}

for key in tqdm.tqdm(persons):
    family_results[key] = family_comp(key, persons)

100%|██████████| 59031/59031 [00:01<00:00, 32851.98it/s]


In [20]:
fields = ("personal_income", "childrens_income", "family_income", "personal_sq", "childrens_sq",\
         "family_sq", "personal_veh", "childrens_veh", "family_veh", "ap_sq", "land_sq")

for field in fields:
    max_val = 0
    max_id = 0
    max_year = 0
    for pid in family_results:
        person = family_results[pid]
        for year in person["years"]:
            if person["years"][year][field] > max_val and person["region"]['id']==1:
                max_val = person["years"][year][field]
                max_id = person["id"]
                max_year = year
              
    print("max " + field + " is " + str(max_val) + " belongs to " + persons[str(max_id)]["name"] + "  " + str(max_id) + " " + str(max_year) + " at " + persons[str(max_id)]["region"]["name"]) 

max personal_income is 843596516.76 belongs to Рассудов Александр Николаевич  32283 2017 at Санкт-Петербург
max childrens_income is 14490000.0 belongs to Шушвал Максим Николаевич  58550 2018 at Санкт-Петербург
max family_income is 276495179.52 belongs to Кичеджи Василий Николаевич  999 2016 at Санкт-Петербург
max personal_sq is 86309054.1 belongs to Чапаева Елена Андреевна  53372 2016 at Санкт-Петербург
max childrens_sq is 50196.0 belongs to Филиппов К. А.  35408 2014 at Санкт-Петербург
max family_sq is 52072785.99999999 belongs to Лысакова Ирина Васильевна  54746 2016 at Санкт-Петербург
max personal_veh is 19 belongs to Антошевский Александр Константинович  58951 2018 at Санкт-Петербург
max childrens_veh is 1 belongs to Фролова Юлия Геннадьевна  6486 2016 at Санкт-Петербург
max family_veh is 11 belongs to Васильева Людмила Александровна  54526 2016 at Санкт-Петербург
max ap_sq is 3076984.0 belongs to Легкая Лидия Сергеевна  9367 2018 at Санкт-Петербург
max land_sq is 104145297.6 belon

In [14]:
new_family_results = {}

for pid in family_results:
    if family_results[pid]["region"]["id"] == 1 or family_results[pid]["region"]["id"] == 63:
        new_family_results[pid] = family_results[pid]
family_results = new_family_results

In [18]:
with open("family_summ_with_reg.json", "w") as f:
    json.dump(family_results, f)

In [53]:
import pandas as pd


In [61]:
family_results_year = {}
year = 2018
for key in family_results:
    if year in family_results[key]["years"]:
        family_results_year[key] = {
            "name": family_results[key]["name"],
            "id": family_results[key]["id"]
        }
        for field in fields:
            family_results_year[key][field] = family_results[key]["years"][year][field]

In [71]:
df = pd.DataFrame.from_dict(family_results_year, orient="index")

In [72]:
df.head()

,name,id,personal_income,childrens_income,family_income,personal_sq,childrens_sq,family_sq,personal_veh,childrens_veh,family_veh,ap_sq,land_sq
100,Островский Алексей Владимирович,100,3.392518e+06,0.0,0.00,3212.2,68.0,34.0,1,0,0,1182.2,2132.0
10032,Большаков Илья Владимирович,10032,1.879366e+06,0.0,252686.18,205.9,253.2,149.7,2,0,0,608.8,0.0
100437,Бородовицын Андрей Николаевич,100437,1.546490e+06,0.0,2216340.71,850.4,761.8,761.8,2,0,0,366.1,2007.9
1007,Пономарев Валерий Андреевич,1007,2.345810e+09,0.0,27070526.61,603.3,0.0,188.0,4,0,0,680.5,110.8
10080,Афанасьева Ирина Кирилловна,10080,1.535041e+06,0.0,0.00,60.6,0.0,0.0,0,0,0,60.6,0.0


In [46]:
family_results["8"]["years"][1999]["personal_income"]

495443.0